<a href="https://colab.research.google.com/github/rahiakela/deep-learning-research-and-practice/blob/main/math-and-architectures-of-deep-learning/02-introduction-to-vectors-calculus/01_gradient_descent_computation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Gradient Descent Computation

In machine learning, we identify the input and output variables
pertaining to the problem at hand and cast the problem as
generating outputs from input variables. All the inputs are
represented together by the vector $\vec{x}$. Sometimes there
are multiple outputs, sometimes single output. Accordingly,
we have an output vector $\vec{y}$ or output scalar $y$.
Let us denote the function that generates the output from input
 vector as $f$, i.e., $y = f\left(\vec{x}\right)$.

In real life problems, we do not know $f$. The crux of machine
learning is to estimate $f$ from a set of observed inputs
$\vec{x}_{i}$ and their corresponding outputs $y_{i}$.
Each observation can be depicted as a pair $\langle\vec{x}_{i}, y_{i}\rangle$.
We model the unknown function $f$ with a known function $\phi$.
$\phi$ is a parameterized function. Alhtough the nature of $\phi$
is known, its parameter values are unknown. These parameter values
 are "learnt" via training. This means, we estimate the parameter
values such that the overall error on the observations is minimized.

If $\vec{w}, b$ denotes the current set of parameters (weights, bias), then the model will
output $\phi\left(\vec{x}_{i}, \vec{w}, b\right)$ on the observed input $\vec{x}_{i}$.
Thus the error on this $i^{th}$ observation is $e_{i}^{2}=\left(\phi\left(\vec{x}_{i}, \vec{w}, b\right) - y_{i}\right)^{2}$.
We can batch up several observations and add up the error into a batch error
$L = \sum_{i=0}^{i=N}\left(e^{\left(i\right)}\right)^{2}$.

The error is a function of the parameter set $\vec{w}$.
The question is: how do we adjust $\vec{w}$ so that the error $e_{i}^{2}$ decreases.
We know a function's value changes most when we move along the direction of
of the gradient of the parameters. Hence, we adjust the parameters
$\vec{w}, b$ as
$\begin{bmatrix}
\vec{w}\\b
\end{bmatrix} = \begin{bmatrix}
\vec{w}\\b
\end{bmatrix} - \mu \nabla_{\vec{w}, b}L\left(\vec{w}, b\right)$.
Each adjustment reduces the error. Starting from a random set of parameter values
doing this "sufficiently" large number of times yields the desired model.

A simple and popular model $\phi$ is the linear function (predicted value is
dot product between input and parameters plus bias):
$\tilde{y}_{i} = \phi\left(\vec{x}_{i}, \vec{w}, b\right) = \vec{w}^{T}\vec{x} + b
= \sum_{j}w_{j}x_{j} + b$.
In the example below, this is the model architecture used.

Thus 
\begin{align*}
L &= \sum_{i=0}^{i=N}\left(e^{\left(i\right)}\right)^{2}\\
  &= \sum_{i=0}^{i=N}\left(\vec{w}^{T}\vec{x} + b - y_{i}\right)^{2}\\
\nabla_{\vec{w}, b}L &\propto \sum_{i=0}^{i=N}\left(\vec{w}^{T}\vec{x}_{i} + b - y_{i}\right)\vec{x}_{i} \\
                     &\propto \sum_{i=0}^{i=N}\left(\tilde{y}_{i} - y_{i}\right)\vec{x}_{i}
\end{align*}
Our initial implementation will simply mimic this formula.
For more complicated models $\phi$ (with millions of parameters and non-linearities)
we cannot obtain closed form gradients like this.

The next example, based on NumPy and PyTorch, relies on PyTorch's
autograd (automatic gradient computation) which does not have this limitation.

##Setup

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt

In [2]:
def update_parameters(params, learning_rate):
  """
  Update the current weight and bias values
  from gradient values.
  """
  # Don't track gradients while updating params
  with torch.no_grad():
      for i, p in enumerate(params):
          params[i] = p - learning_rate * p.grad
          
  # Restore tracking of gradient for all params
  for i in range(len(params)):
      params[i].requires_grad = True


def draw_line(m, c, min_x=0, max_x=10, color='magenta', label=None):
  """
  Plots y = mx + c from interval (min_x to max_x)
  """
  # linspace creates an array of equally spaced
  # values between the specified min and max in 
  # specified number of steps.
  x = np.linspace(min_x, max_x, 100)
  y = m*x + c
  
  plt.plot(x, y, color=color, 
            label='y=%0.2fx+%0.2f'%(m, c)\
                if not label else label)
    
def draw_parabola(w0, w1, w2,  min_x=0, max_x=10, color='magenta', label=None):
  """
  Plots y = w0 + w1*x + w2*x^2 from interval
  (min_x to max_x)
  """
  x = np.linspace(min_x, max_x, 100)
  y = w0 + w1*x +  w2* (x**2)
  plt.plot(x, y, color=color, 
            label='y=%0.2f+ %0.2fx + %0.2fx^2'
            %(w0, w1, w2) if not label else label)
    
def draw_subplot(pos, step, true_draw_func, true_draw_params, pred_draw_func, pred_draw_params):
  """
  Plots the curves corresponding to a specified pair of functions.
  We use it to plot
  (i) the true function (used to generate the observations
      that we are trying to predict with a trained mode) vis a vis
  (ii) the model function (used to makes the predictions)
        When the predictor is good, the two plots should more or less coincide.
  Thus this is used to visualize the goodness of the current approximation.
  """
  plt.subplot(2, 2, pos)
  plt.title('Step %d'%(step))
  true_draw_func(**true_draw_params)
  pred_draw_func(**pred_draw_params)
  plt.xlabel('x')
  plt.ylabel('y')
  plt.legend(loc='upper left')

## Gradient Descent (manually) via NumPy

In [3]:
np.random.seed(0)

# generate random input values
N = 100
x = 10 * np.random.randn(N)

# Compute function outputs
y = 1.5 * x + 2.73

# Add random noise to get the observed value of y
y_obs = y + (0.5 * np.random.randn(N))

# Random initialization of model parameters
w = np.random.randn(1)
b = np.random.randn(1)

# Training: repeatative adjustment of parameters via gradient.
num_steps = 4000
learning_rate = 1e-3

for step in range(num_steps):
  # our model - initialized with arbitrary parameter values
  y_pred = w * x + b
  mean_squared_error = np.mean(y_pred - y_obs) ** 2

  # Gradient of error computation using calculus formulas.
  w_grad = np.mean(2 * ((y_pred - y_obs) * x))
  b_grad = np.mean(2 * (y_pred - y_obs))
  w = w - learning_rate * w_grad
  b = b - learning_rate * b_grad

  # periodically print diagnostic messages
  if step % 400 == 0:
    print(f"Step {step}: w = {w} b = {b} \nMSE Error = {mean_squared_error}")
    print(f"Gradient of w: {w_grad} \nGradient of b: {b_grad}")

print("True function: y = 1.5*x + 2.73")
print(f"Learnt function: y_pred = {w[0]}*x + {b[0]}")

Step 0: w = [0.01667435] b = [-0.23112257] 
MSE Error = 17.042912303509105
Gradient of w: -385.85618670584734 
Gradient of b: -8.256612453908469
Step 400: w = [1.5137106] b = [1.42144298] 
MSE Error = 1.8064489836696587
Gradient of w: 0.015926565641006717 
Gradient of b: -2.6880840639159027
Step 800: w = [1.50932594] b = [2.16148491] 
MSE Error = 0.36620643215003873
Gradient of w: 0.007170876936491713 
Gradient of b: -1.2102998506982288
Step 1200: w = [1.50735176] b = [2.49468604] 
MSE Error = 0.07423799518303086
Gradient of w: 0.003228660666584702 
Gradient of b: -0.5449330057283404
Step 1600: w = [1.5064629] b = [2.64470861] 
MSE Error = 0.015049653542234021
Gradient of w: 0.0014536924552342434 
Gradient of b: -0.24535405879857802
Step 2000: w = [1.50606269] b = [2.71225571] 
MSE Error = 0.003050891543917218
Gradient of w: 0.0006545196205436543 
Gradient of b: -0.11046975231106872
Step 2400: w = [1.5058825] b = [2.74266854] 
MSE Error = 0.0006184819595096562
Gradient of w: 0.00029469

## Gradient Descent (manually) via PyTorch

In [4]:
torch.manual_seed(42)

# generate random input values
N = 100
x = 10 * torch.randn(N)

# Compute function outputs
y = 1.5 * x + 2.73

# Add random noise to get the observed value of y
y_obs = y + (0.5 * torch.randn(N))

# Random initialization of model parameters
w = torch.randn(1)
b = torch.randn(1)

# Training: repeatative adjustment of parameters via gradient.
num_steps = 4000
learning_rate = 1e-3

for step in range(num_steps):
  # our model - initialized with arbitrary parameter values
  y_pred = w * x + b
  mean_squared_error = torch.mean(y_pred - y_obs) ** 2

  # Gradient of error computation using calculus formulas.
  w_grad = torch.mean(2 * ((y_pred - y_obs) * x))
  b_grad = torch.mean(2 * (y_pred - y_obs))
  w = w - learning_rate * w_grad
  b = b - learning_rate * b_grad

  # periodically print diagnostic messages
  if step % 400 == 0:
    print(f"Step {step}: w = {w} b = {b} \nMSE Error = {mean_squared_error}")
    print(f"Gradient of w: {w_grad} \nGradient of b: {b_grad}")

print("True function: y = 1.5*x + 2.73")
print(f"Learnt function: y_pred = {w[0]}*x + {b[0]}")

Step 0: w = tensor([0.6108]) b = tensor([-0.2075]) 
MSE Error = 13.136869430541992
Gradient of w: -217.66844177246094 
Gradient of b: -7.248963832855225
Step 400: w = tensor([1.5089]) b = tensor([1.4195]) 
MSE Error = 1.7582067251205444
Gradient of w: 0.016568755730986595 
Gradient of b: -2.6519477367401123
Step 800: w = tensor([1.5043]) b = tensor([2.1497]) 
MSE Error = 0.3565356135368347
Gradient of w: 0.007476482540369034 
Gradient of b: -1.1942120790481567
Step 1200: w = tensor([1.5023]) b = tensor([2.4785]) 
MSE Error = 0.07230029255151749
Gradient of w: 0.0033882809802889824 
Gradient of b: -0.5377742648124695
Step 1600: w = tensor([1.5013]) b = tensor([2.6265]) 
MSE Error = 0.014661334455013275
Gradient of w: 0.0015091609675437212 
Gradient of b: -0.24216799437999725
Step 2000: w = tensor([1.5009]) b = tensor([2.6932]) 
MSE Error = 0.0029730559326708317
Gradient of w: 0.0006666564731858671 
Gradient of b: -0.10905147343873978
Step 2400: w = tensor([1.5007]) b = tensor([2.7232]) 